In [ ]:
import os
import shutil
import glob

# 0. Hapus folder PDF lama (jika ada) untuk memastikan bersih
folder_name = "pdf_daftar_peserta_landscape"
if os.path.exists(folder_name):
    print(f"Folder lama '{folder_name}' ditemukan. Menghapusnya...")
    shutil.rmtree(folder_name)
    print(f"Folder '{folder_name}' telah dihapus.")
else:
    print(f"Folder '{folder_name}' tidak ditemukan, akan dibuat baru nanti.")
print("\n" + "="*50 + "\n")

# 1. Instal modul reportlab terlebih dahulu
!pip install reportlab

# 2. Impor library setelah instalasi
import pandas as pd
from google.colab import files
import io
from reportlab.lib.pagesizes import A4, landscape
from reportlab.platypus import SimpleDocTemplate, Table, TableStyle, Paragraph, Spacer, PageBreak, Image
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle, TA_LEFT, TA_CENTER, TA_RIGHT
from reportlab.lib import colors
from reportlab.lib.units import inch
from reportlab.lib.utils import simpleSplit

print("--- Langkah 1: Upload File Excel ---")
print("Silakan upload file Excel Manajemen Siswa Anda.")
uploaded_excel = files.upload()

if not uploaded_excel:
    print("Tidak ada file Excel yang diupload. Menghentikan proses.")
    exit()

excel_file_name = list(uploaded_excel.keys())[0]
print(f"File Excel '{excel_file_name}' telah diupload.")
print("\n" + "="*50 + "\n")

print("--- Langkah 2: Upload Logo Bebras dan USU (opsional) ---")
print("Silakan upload file gambar logo Bebras Indonesia (misalnya: logo_bebras.png).")
uploaded_bebras_logo = files.upload()
if not uploaded_bebras_logo:
    print("Tidak ada file logo Bebras yang diupload. Proses akan dilanjutkan tanpa logo Bebras.")
    bebras_logo_path = None
else:
    bebras_logo_path = list(uploaded_bebras_logo.keys())[0]
    print(f"File logo Bebras '{bebras_logo_path}' telah diupload.")

print("\nSilakan upload file gambar logo Universitas Sumatera Utara (misalnya: logo_usu.png).")
uploaded_usu_logo = files.upload()
if not uploaded_usu_logo:
    print("Tidak ada file logo USU yang diupload. Proses akan dilanjutkan tanpa logo USU.")
    usu_logo_path = None
else:
    usu_logo_path = list(uploaded_usu_logo.keys())[0]
    print(f"File logo USU '{usu_logo_path}' telah diupload.")

print("\n" + "="*50 + "\n")

# 3. Baca file Excel
df_raw = pd.read_excel(io.BytesIO(uploaded_excel[excel_file_name]))

# Perbaiki nama kolom: hapus spasi di awal/akhir
df = df_raw.copy()
df.columns = df.columns.str.strip()

print("Nama kolom dalam DataFrame:")
print(df.columns.tolist()) # Debug: Tampilkan nama kolom yang telah dibersihkan
print("\n" + "="*50 + "\n")

# 4. Dapatkan semua kombinasi unik sekolah, pendamping, dan kode verifikasi
# Ambil 4 digit terakhir dari 'No Kontak Pendamping' sebagai kode verifikasi
df['verification_code'] = df['No Kontak Pendamping'].astype(str).str[-4:]

# Kelompokkan berdasarkan Sekolah untuk membuat file PDF per sekolah
# Kita akan iterasi berdasarkan 'Sekolah' unik
df_sekolah_grouped = df.groupby('Sekolah')

print("Daftar Sekolah Unik:")
for idx, sekolah in enumerate(df_sekolah_grouped.groups.keys(), 1):
    print(f"{idx:3d}. {sekolah}")
print(f"\nTotal Sekolah: {len(df_sekolah_grouped)}")
print("\n" + "="*50 + "\n")

# 5. Buat folder untuk menyimpan PDF (baru)
os.makedirs(folder_name, exist_ok=True)
print(f"Membuat folder: {folder_name}")

# 6. Fungsi untuk membuat PDF untuk satu sekolah (mencakup semua pendamping di sekolah tersebut)
def buat_pdf_per_sekolah(df_sekolah_group, nama_sekolah):
    # Periksa apakah grup siswa kosong
    if df_sekolah_group.empty:
        print(f"Peringatan: Tidak ditemukan data siswa untuk sekolah '{nama_sekolah}'. Melewati pembuatan PDF.")
        return # Kembali jika tidak ada data siswa

    # Siapkan nama file PDF berdasarkan Nama Sekolah
    clean_nama_sekolah = "".join(c for c in nama_sekolah if c.isalnum() or c in (' ', '-', '_')).rstrip()
    pdf_filename = os.path.join(folder_name, f"Daftar_Peserta_{clean_nama_sekolah.replace(' ', '_')}.pdf")

    # Buat dokumen PDF dengan orientasi landscape A4
    doc = SimpleDocTemplate(pdf_filename, pagesize=landscape(A4), topMargin=0.5*inch, bottomMargin=0.5*inch, leftMargin=0.5*inch, rightMargin=0.5*inch)
    story = []

    # Gaya teks
    title_style = ParagraphStyle(
        'CustomTitle',
        fontSize=18,
        leading=22,
        spaceAfter=12,
        alignment=TA_CENTER,
        textColor=colors.darkblue,
        fontName='Helvetica-Bold'
    )
    heading_style = ParagraphStyle(
        'CustomHeading',
        fontSize=12,
        leading=14,
        spaceAfter=6,
        textColor=colors.black,
        fontName='Helvetica-Bold'
    )
    normal_style = getSampleStyleSheet()['Normal']
    normal_style.fontSize = 10
    normal_style.leading = 12
    normal_style.alignment = TA_LEFT # Rata kiri untuk teks biasa

    # Gaya untuk subjudul jadwal
    schedule_title_style = ParagraphStyle(
        'ScheduleTitle',
        parent=normal_style,
        fontSize=14,
        leading=16,
        spaceAfter=6,
        fontName='Helvetica-Bold',
        textColor=colors.darkgreen,
        alignment=TA_LEFT
    )

    # Gaya untuk bagian informasi jadwal
    schedule_info_style = ParagraphStyle(
        'ScheduleInfo',
        parent=normal_style,
        fontSize=9,
        leading=11,
        spaceAfter=4,
        alignment=TA_LEFT
    )

    # Gaya untuk tabel jadwal
    schedule_table_style = ParagraphStyle(
        'ScheduleTable',
        parent=normal_style,
        fontSize=8,
        leading=10,
        alignment=TA_LEFT
    )

    # Gaya untuk subjudul Tanya Jawab
    faq_title_style = ParagraphStyle(
        'FAQTitle',
        parent=normal_style,
        fontSize=14,
        leading=16,
        spaceAfter=6,
        fontName='Helvetica-Bold',
        textColor=colors.darkred,
        alignment=TA_LEFT
    )

    # Gaya untuk bagian Tanya Jawab
    faq_info_style = ParagraphStyle(
        'FAQInfo',
        parent=normal_style,
        fontSize=9,
        leading=11,
        spaceAfter=4,
        alignment=TA_LEFT
    )


    # --- Bagian Atas PDF (Kop Surat) ---
    # Siapkan elemen kop surat: Logo Bebras di kiri, Judul di tengah, Logo USU di kanan
    kop_surat_elements = []
    try:
        if bebras_logo_path:
            # Muat gambar dan atur tinggi maksimum agar konsisten dengan logo USU
            bebras_logo_img = Image(bebras_logo_path, width=1.5*inch, height=1.2*inch, kind='proportional') # Tinggi diset ke 1.2 inch
            kop_surat_elements.append(bebras_logo_img)
        else:
            kop_surat_elements.append(Paragraph("", normal_style)) # Placeholder jika tidak ada logo
    except Exception as e:
        print(f"Error memuat logo Bebras: {e}")
        kop_surat_elements.append(Paragraph("", normal_style))

    # Judul sebagai link
    title_text = '<a href="https://bebras.or.id/v3/bebras-indonesia-challenge-2025/" color="darkblue"><u>DAFTAR PESERTA BEBRAS CHALLENGE 2025</u></a>'
    title_para = Paragraph(title_text, title_style)
    kop_surat_elements.append(title_para)

    try:
        if usu_logo_path:
            # Muat gambar dan atur tinggi maksimum agar konsisten dengan logo Bebras
            usu_logo_img = Image(usu_logo_path, width=1.5*inch, height=1.2*inch, kind='proportional') # Tinggi diset ke 1.2 inch
            kop_surat_elements.append(usu_logo_img)
        else:
            kop_surat_elements.append(Paragraph("", normal_style)) # Placeholder jika tidak ada logo
    except Exception as e:
        print(f"Error memuat logo USU: {e}")
        kop_surat_elements.append(Paragraph("", normal_style))

    # Buat tabel kop surat
    # Kolom: Logo Bebras (lebar 1.5 inch), Judul (lebar tersisa), Logo USU (lebar 1.5 inch)
    available_width = doc.width
    kop_surat_col_widths = [1.5*inch, available_width - 3*inch, 1.5*inch] # Total lebar harus sama dengan doc.width
    kop_surat_table_data = [kop_surat_elements]
    kop_surat_table = Table(kop_surat_table_data, colWidths=kop_surat_col_widths)
    kop_surat_table.setStyle(TableStyle([
        ('ALIGN', (0, 0), (-1, -1), 'CENTER'),
        ('VALIGN', (0, 0), (-1, -1), 'MIDDLE'), # Sejajarkan vertikal ke tengah
        ('TOPPADDING', (0, 0), (-1, -1), 0),
        ('BOTTOMPADDING', (0, 0), (-1, -1), 0),
        ('LEFTPADDING', (0, 0), (-1, -1), 0),
        ('RIGHTPADDING', (0, 0), (-1, -1), 0),
    ]))

    story.append(kop_surat_table)
    story.append(Spacer(1, 6)) # Spasi setelah kop surat


    # Tambahkan informasi sekolah, pendamping, kontak
    story.append(Paragraph(f"<b>Sekolah:</b> {nama_sekolah}", heading_style))
    story.append(Paragraph(f"<b>Kota/Kabupaten:</b> {df_sekolah_group['Kota/ Kabupaten'].iloc[0]}", normal_style))

    # Ambil pendamping dan kontak unik dari grup ini (karena grup berdasarkan Sekolah)
    # Gunakan dropna() untuk menghindari entri kosong
    pendamping_list = df_sekolah_group['Pendamping'].dropna().unique()
    kontak_list = df_sekolah_group['No Kontak Pendamping'].dropna().unique()
    kode_verifikasi_list = df_sekolah_group['verification_code'].dropna().unique()

    # Gabungkan pendamping jika lebih dari satu
    pendamping_str = ", ".join(pendamping_list)
    # Gabungkan kontak jika lebih dari satu
    kontak_formatted_list = []
    for kontak_asli in kontak_list:
        kontak_str = str(kontak_asli).strip()
        if kontak_str and kontak_str.startswith('0'):
            kontak_str = '+62' + kontak_str[1:] # Ganti 0 di awal dengan +62
        elif kontak_str and not kontak_str.startswith('+62'):
             kontak_str = '+62' + kontak_str # Jika tidak diawali 0 atau +62, tambahkan +62 di awal
        # Buat link WhatsApp Web
        wa_link = f'<a href="https://wa.me/{kontak_str.replace("+", "")}" color="blue">{kontak_str}</a>' # Hapus + sebelum kirim ke wa.me
        kontak_formatted_list.append(wa_link)
    formatted_kontak_str = ", ".join(kontak_formatted_list)

    story.append(Paragraph(f"<b>Pendamping:</b> {pendamping_str}", heading_style))
    story.append(Paragraph(f"<b>Kontak Pendamping:</b> {formatted_kontak_str}", heading_style))
    story.append(Spacer(1, 12)) # Spasi sebelum tabel


    # --- Tabel Peserta (Diperbarui) ---
    # Header tabel diperbarui
    table_data = [["No", "Nama Peserta", "Username", "Password", "Sesi", "Kelas", "Jenjang"]]
    # Iterasi dan tambahkan data peserta, nomor dimulai dari 1
    for idx, (row_idx, row) in enumerate(df_sekolah_group.iterrows(), start=1):
        table_data.append([
            str(idx), # Gunakan indeks lokal mulai dari 1
            row['Nama Lengkap'],
            row['Username'],
            row['Password'],
            row['Sesi'],
            str(row['Kelas']), # Konversi ke string untuk keamanan
            row['Jenjang']
        ])

    # Hitung lebar kolom secara proporsional terhadap lebar halaman
    available_width = doc.width # Ambil lebar efektif dari dokumen
    col_widths = [0.06, 0.25, 0.15, 0.15, 0.06, 0.06, 0.07] # Proporsi kolom
    col_widths = [w * available_width for w in col_widths] # Kalikan dengan lebar total

    table = Table(table_data, colWidths=col_widths)
    table.setStyle(TableStyle([
        # Gaya Header
        ('BACKGROUND', (0, 0), (-1, 0), colors.steelblue),
        ('TEXTCOLOR', (0, 0), (-1, 0), colors.whitesmoke),
        ('ALIGN', (0, 0), (-1, 0), 'CENTER'),
        ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
        ('FONTSIZE', (0, 0), (-1, 0), 10), # Ukuran font header
        ('BOTTOMPADDING', (0, 0), (-1, 0), 8),
        # Gaya isi tabel
        ('BACKGROUND', (0, 1), (-1, -1), colors.beige),
        ('FONTNAME', (0, 1), (-1, -1), 'Helvetica'),
        ('FONTSIZE', (0, 1), (-1, -1), 8), # Ukuran font isi tabel diperbesar
        ('GRID', (0, 0), (-1, -1), 1, colors.black),
        ('VALIGN', (0, 0), (-1, -1), 'MIDDLE'),
        ('ALIGN', (0, 0), (-1, -1), 'CENTER'),
        ('ALIGN', (1, 0), (1, -1), 'LEFT'), # Nama Peserta rata kiri
        ('ALIGN', (5, 0), (5, -1), 'CENTER'), # Kelas rata tengah
        ('ALIGN', (6, 0), (6, -1), 'CENTER'), # Jenjang rata tengah
    ]))

    story.append(table)
    story.append(Spacer(1, 12)) # Spasi sebelum informasi jadwal


    # --- Informasi Jadwal dan Aturan (Diperbarui dan digabungkan) ---
    # Subjudul
    schedule_title = Paragraph("Jadwal & Pelaksanaan Tantangan Bebras 2025", schedule_title_style)
    story.append(schedule_title)
    story.append(Spacer(1, 6)) # Spasi setelah subjudul

    # Informasi jadwal dan aturan lengkap digabungkan
    jadwal_utama_baru = """
    <b>1. Jadwal pelaksanaan BC2025 adalah sebagai berikut:</b><br/>
    <b>Coba Akun</b> dilaksanakan pada,<br/>
    &nbsp;&nbsp;&nbsp;&nbsp;•&nbsp;Mulai: Minggu, 9 November 2025, pukul 08.00<br/>
    &nbsp;&nbsp;&nbsp;&nbsp;•&nbsp;Berakhir: Senin, 10 November 2025, pukul 21.00<br/><br/>
    Peserta dapat menggunakan username dan password yang telah dibagikan untuk mencoba login dengan akun mereka, dan mengerjakan 3 soal untuk pemanasan, sesuai dengan kategori dan bahasa yang dipilih.<br/><br/>
    <b>2. Pelaksanaan BC2025 dilaksanakan dengan jadwal:</b><br/>
    &nbsp;&nbsp;&nbsp;&nbsp;•&nbsp;<b>Kategori Penegak:</b> Selasa, 11 November 2025<br/>
    &nbsp;&nbsp;&nbsp;&nbsp;•&nbsp;<b>Kategori Penggalang:</b> Rabu, 12 November 2025<br/>
    &nbsp;&nbsp;&nbsp;&nbsp;•&nbsp;<b>Kategori Siaga:</b> Kamis, 13 November 2025<br/>
    &nbsp;&nbsp;&nbsp;&nbsp;•&nbsp;<b>Kategori SiKecil:</b> Jumat, 14 November 2025<br/><br/>
    <b>Sesi:</b><br/>
    &nbsp;&nbsp;&nbsp;&nbsp;•&nbsp;Sesi A: pukul 07.00 – 08.15 WIB<br/>
    &nbsp;&nbsp;&nbsp;&nbsp;•&nbsp;Sesi B: pukul 08.15 – 09.30 WIB<br/>
    &nbsp;&nbsp;&nbsp;&nbsp;•&nbsp;Sesi C: pukul 09.30 – 10.45 WIB<br/>
    &nbsp;&nbsp;&nbsp;&nbsp;•&nbsp;Sesi D: pukul 10.45 – 12.00 WIB<br/>
    &nbsp;&nbsp;&nbsp;&nbsp;•&nbsp;Sesi E: pukul 12.00 – 13.15 WIB<br/>
    &nbsp;&nbsp;&nbsp;&nbsp;•&nbsp;Sesi F: pukul 13.15 – 14.30 WIB<br/>
    &nbsp;&nbsp;&nbsp;&nbsp;•&nbsp;Sesi G: pukul 14.30 – 15.45 WIB<br/>
    &nbsp;&nbsp;&nbsp;&nbsp;•&nbsp;Sesi H: pukul 15.45 – 17.00 WIB<br/>
    &nbsp;&nbsp;&nbsp;&nbsp;•&nbsp;Sesi I: pukul 17.00 – 18.15 WIB<br/><br/>
    Peserta dapat mengikuti BC2025, sesuai jadwal hari dan sesinya.<br/>
    Login pada sistem <a href="https://tantanganbebras.ipb.ac.id" color="blue">https://tantanganbebras.ipb.ac.id</a> dibatasi sesuai dengan hari dan kategori yang dipilih,<br/>
    Pengerjaan Tantangan dibatasi sesuai dengan hari dan sesi per kategori-bahasa.<br/><br/>
    <b>3. Pra Tantangan</b><br/>
    &nbsp;&nbsp;&nbsp;&nbsp;•&nbsp;Setiap peserta diharapkan mencoba untuk mengakses situs tantangan di <a href="https://tantanganbebras.ipb.ac.id" color="blue">https://tantanganbebras.ipb.ac.id</a> untuk mencoba akun dan berlatih kecil sebelum jadwal tantangan pada hari 9 – 10 November 2025.<br/>
    &nbsp;&nbsp;&nbsp;&nbsp;•&nbsp;Coba Akun/Latihan di situs tantangan diberikan 3 kali kesempatan (attempt).<br/>
    &nbsp;&nbsp;&nbsp;&nbsp;•&nbsp;Peserta diharapkan mempersiapkan perangkat dan koneksi internet yang cukup baik saat lomba untuk menghindari kesulitan saat lomba (misalnya: baterai laptop/hp penuh, koneksi tidak putus, dll).<br/><br/>
    <b>4. Saat Tantangan</b><br/>
    &nbsp;&nbsp;&nbsp;&nbsp;•&nbsp;Setiap peserta mengikuti tantangan sesuai dengan jadwal yang telah ditentukan sesuai dengan kategori (SiKecil, Siaga, Penggalang, Penegak) dan bahasa (bahasa Indonesia atau bahasa Inggris).<br/>
    &nbsp;&nbsp;&nbsp;&nbsp;•&nbsp;Tantangan hanya diperbolehkan 1 kali kesempatan (attempt).<br/>
    &nbsp;&nbsp;&nbsp;&nbsp;•&nbsp;Persiapkan Kertas dan Pensil.<br/>
    &nbsp;&nbsp;&nbsp;&nbsp;•&nbsp;Tantangan dapat dilaksanakan secara luring.<br/>
    &nbsp;&nbsp;&nbsp;&nbsp;•&nbsp;Jika ada gangguan, guru dapat melaporkan gangguan tersebut ke Biro Bebras yang menaunginya, yang selanjutnya akan ditangani secara terpisah.<br/><br/>
    <b>5. Pengumuman Hasil</b><br/>
    &nbsp;&nbsp;&nbsp;&nbsp;•&nbsp;Hasil Tantangan Bebras akan dipublikasi setelah semua keluhan/permasalahan selesai ditangani dan data diproses.<br/>
    &nbsp;&nbsp;&nbsp;&nbsp;•&nbsp;Hasil Tantangan dipublikasi di laman <a href="http://bebras.or.id" color="blue">http://bebras.or.id</a><br/>
    &nbsp;&nbsp;&nbsp;&nbsp;•&nbsp;National Board Organization (NBO) Bebras akan memberikan sertifikat kepada semua peserta.<br/>
    &nbsp;&nbsp;&nbsp;&nbsp;•&nbsp;Bebras Biro dapat memberikan penghargaan kepada peserta dengan kriteria yang ditetapkan.<br/><br/>
    <b>6. Sertifikat</b><br/>
    Ada 2 jenis sertifikat:<br/>
    &nbsp;&nbsp;&nbsp;&nbsp;a) Sertifikat dengan mencantumkan nilai tantangan (jika nilai >= 50)<br/>
    &nbsp;&nbsp;&nbsp;&nbsp;b) Sertifikat tanda keikutsertaan, yang tidak mencantumkan nilai tantangan (nilai < 50)<br/><br/>
    Sertifikat akan dikirim dalam bentuk e-certificate ke peserta.<br/><br/>
    <b>Catatan:</b> untuk menghindari salah nama atau data lain pada sertifikat, hendaknya guru mencermati data pendaftaran dengan teliti. Biro dan Panitia Pusat tidak bertanggung jawab untuk kesalahan data setelah sertifikat dicetak.<br/><br/>
    <b>7. Jika ada kendala</b> yang disebabkan oleh sistem/koneksi/listrik/gagal login/sesi tidak sesuai/dll peserta dapat mengisi keluhan di <a href="https://bit.ly/FormMasalahBC2025" color="blue">https://bit.ly/FormMasalahBC2025</a>. Tantangan Susulan akan dilaksanakan pada hari Senin, 17 November 2025.<br/><br/>
    Mohon untuk <b>mengedepankan integritas dan kejujuran</b>.<br/><br/>
    <b>8. Grup WhatsApp</b><br/>
    Bagi Bapak/Ibu Pendamping yang belum bergabung ke grup WhatsApp Bebras Challenge Biro USU dapat mengklik <a href="https://chat.whatsapp.com/GKej5mRCFnqIJOcFYRsLQ2?mode=wwt" color="blue">link tersebut</a>.<br/><br/>
    Demikian<br/>
    Terimakasih<br/><br/>
    """

    story.append(Paragraph(jadwal_utama_baru, schedule_info_style))

    # --- Tanya Jawab ---
    # Subjudul
    faq_title = Paragraph("Tanya Jawab (Referensi Pendamping)", faq_title_style)
    story.append(faq_title)
    story.append(Spacer(1, 6)) # Spasi setelah subjudul

    # Informasi Tanya Jawab
    faq_content = """
    <b>Apakah situs yang dipakai untuk tantangan bebras sama dengan situs yang selama ini dipakai untuk latihan?</b> JAWAB: TIDAK.<br/>
    Platform latihan diakses di <a href="https://latihan.bebras.or.id" color="blue">https://latihan.bebras.or.id</a> berbeda dengan Platform Tantangan yang diakses di <a href="https://tantanganbebras.ipb.ac.id" color="blue">https://tantanganbebras.ipb.ac.id</a> dan sengaja dibuat terpisah.<br/>
    Walaupun platform nya berbeda, tampilan dan antarmuka interaksi dengan sistem sama sehingga seharusnya peserta yang sudah pernah berlatih di platform latihan akan sudah terbiasa serta tidak canggung berinteraksi dengan sistem.<br/><br/>

    <b>Bagaimana memperoleh akun di platform Latihan?</b><br/>
    &nbsp;&nbsp;&nbsp;&nbsp;•&nbsp;Kunjungi situs <a href="https://latihan.bebras.or.id" color="blue">https://latihan.bebras.or.id</a>. Di laman tersebut, ada panduan pembuatan akun dalam bentuk file PDF yang dapat diunduh.<br/>
    &nbsp;&nbsp;&nbsp;&nbsp;•&nbsp;User mendaftarkan diri melalui fasilitas “Create New Account”, mendapatkan konfirmasi email, dan melakukan “Enroll” pada latihan yang tersedia. Pilihlah Latihan yang sesuai dengan Kategori yang akan dipilih. Pada Latihan, siswa dapat memilih semua kategori. Jika menjumpai kesulitan, ada baiknya untuk memulai latihan kategori lebih rendah (misalnya akan mengikuti Penggalang, mulai latihan Siaga sebelum mengerjakan Penggalang).<br/>
    &nbsp;&nbsp;&nbsp;&nbsp;•&nbsp;Karena ada batasan usia untuk mempunyai akun email, maka bagi siswa SD, akun dapat dibuatkan dan dikelola oleh orangtua, wali atau guru.<br/><br/>

    <b>Apakah saya harus membuat akun (username dan password) untuk ikut tantangan bebras?</b> JAWAB: TIDAK<br/>
    &nbsp;&nbsp;&nbsp;&nbsp;•&nbsp;User-id dan password untuk tantangan bebras berbeda dengan user-id dan password yang dibuat sendiri (atau diberikan oleh guru) dalam rangka mencoba soal-soal di platform <a href="https://latihan.bebras.or.id" color="blue">https://latihan.bebras.or.id</a>.<br/>
    &nbsp;&nbsp;&nbsp;&nbsp;•&nbsp;Akun tantangan akan dibuatkan oleh sistem dan hanya dapat dipakai saat Coba-Akun dan Tantangan Bebras. Setelah itu, akun tidak bisa dipakai lagi, kecuali untuk masalah yang setelah ditinjau diputuskan oleh Dewan Juri.<br/>
    &nbsp;&nbsp;&nbsp;&nbsp;•&nbsp;User-id dan password akan diberikan oleh guru sekolah kepada siswanya. Guru sekolah mengetahui user-id dan password peserta yang dibinanya.<br/>
    &nbsp;&nbsp;&nbsp;&nbsp;•&nbsp;Guru dan siswa peserta wajib menjaga kerahasiaan password, dan menyatakan “OK” pada laman persetujuan, bahwa tidak akan memberikan password tersebut kepada orang yang tidak berhak.<br/>
    &nbsp;&nbsp;&nbsp;&nbsp;•&nbsp;Khusus untuk akun SiKecil dan SD, guru memberitahukan ke orang tua atau wali peserta yang ditunjuk dan disetujui oleh guru. Orang tua peserta dan wali juga wajib menjaga kerahasiaan password anaknya.<br/>
    &nbsp;&nbsp;&nbsp;&nbsp;•&nbsp;User-id dan password akan dibuat berdasarkan data yang diberikan oleh Guru kepada Panitia Biro Bebras. Oleh karena itu mohon dipastikan agar data benar untuk pencetakan sertifikat.<br/>
    &nbsp;&nbsp;&nbsp;&nbsp;•&nbsp;User-id dan password untuk tantangan hanya dapat dipakai untuk mencoba akses dan latihan mengerjakan soal 3 kali lewat Cek-Akun, dan satu kali saat berpartisipasi pada Tantangan Bebras. Akun yang diberikan hanya dapat dipakai untuk mengakses tantangan Bebras yang dipilih saat pendaftaran (salah satu dari SiKecil/Siaga/Penggalang/Penegak dalam bahasa yang dipilih).<br/><br/>

    <b>Apakah selama Pekan Bebras saya masih bisa latihan soal?</b> JAWAB: YA<br/>
    &nbsp;&nbsp;&nbsp;&nbsp;•&nbsp;Selama pekan bebras, siapapun tetap dapat latihan soal seperti halnya pada hari-hari biasa, di <a href="https://latihan.bebras.or.id" color="blue">https://latihan.bebras.or.id</a>.<br/>
    &nbsp;&nbsp;&nbsp;&nbsp;•&nbsp;Create Akun (jika belum punya) dengan mendaftarkan email yang aktif. Email harus aktif karena Sistem IPB akan mengirimkan email konfirmasi ke email aktif tsb.<br/>
    &nbsp;&nbsp;&nbsp;&nbsp;•&nbsp;Enroll ke latihan yang dipilih (jika sudah punya akun).<br/><br/>

    <b>Soal Tantangan Bebras internasional diterjemahkan ke dalam bahasa Indonesia. Namun, atas permintaan, sejak 2020 tersedia juga dalam bahasa Inggris. Jika saya memilih ikut Bahasa Indonesia, apakah saya bisa mengakses soal bahasa Inggris?</b> atau sebaliknya, jika memilih Bahasa Inggris, apakah bisa mengakses soal bahasa Indonesia? JAWAB: TIDAK, peserta hanya bisa mengakses soal dalam bahasa yang dipilih.<br/>
    &nbsp;&nbsp;&nbsp;&nbsp;•&nbsp;Di situs LATIHAN (<a href="https://latihan.bebras.or.id" color="blue">https://latihan.bebras.or.id</a>) pengguna bebas untuk latihan bahasa Indonesia dan/atau bahasa Inggris, asalkan melakukan “Enroll”.<br/>
    &nbsp;&nbsp;&nbsp;&nbsp;•&nbsp;Di situs TANTANGAN (<a href="https://tantanganbebras.ipb.ac.id" color="blue">https://tantanganbebras.ipb.ac.id</a>), peserta hanya boleh memilih salah satu bahasa, dan hanya dapat mengerjakan soal dalam bahasa yang dipilihnya.<br/><br/>
    """

    story.append(Paragraph(faq_content, faq_info_style))


    # Build PDF
    doc.build(story)
    print(f"PDF untuk sekolah '{nama_sekolah}' telah dibuat: {pdf_filename}")

# 7. Iterasi melalui setiap sekolah unik dan buat PDF
print("--- Memulai Pembuatan File PDF ---")
for nama_sekolah, df_sekolah_ini in df_sekolah_grouped:
    buat_pdf_per_sekolah(df_sekolah_ini, nama_sekolah)

print("\n--- Pembuatan Semua File PDF Selesai ---")
print(f"File-file PDF disimpan di folder: {folder_name}")

# 8. Zip folder dan download
print("\n--- Mengarsipkan dan Mengunduh File ---")
zip_filename = f"{folder_name}.zip"
!zip -r {zip_filename} {folder_name}

print(f"\nMemulai download: {zip_filename}")
files.download(zip_filename)

print("\n--- Proses Selesai ---")
print("Folder PDF telah diunduh sebagai file zip.")

Folder 'pdf_daftar_peserta_landscape' tidak ditemukan, akan dibuat baru nanti.


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 19.6 MB/s eta 0:00:00
--- Langkah 1: Upload File Excel ---
Silakan upload file Excel Manajemen Siswa Anda.


Saving Manajemen Siswa - BebrasWeb (1).xlsx to Manajemen Siswa - BebrasWeb (1).xlsx
File Excel 'Manajemen Siswa - BebrasWeb (1).xlsx' telah diupload.


--- Langkah 2: Upload Logo Bebras dan USU (opsional) ---
Silakan upload file gambar logo Bebras Indonesia (misalnya: logo_bebras.png).


Saving logo bebras.png to logo bebras.png
File logo Bebras 'logo bebras.png' telah diupload.

Silakan upload file gambar logo Universitas Sumatera Utara (misalnya: logo_usu.png).


Saving logo usu.png to logo usu.png
File logo USU 'logo usu.png' telah diupload.


Nama kolom dalam DataFrame:
['Username', 'Nama Lengkap', 'Gender', 'Email', 'Password', 'Sekolah', 'Kota/ Kabupaten', 'Pendamping', 'Jenjang', 'Kelas', 'Bahasa', 'Kategori', 'Sesi', 'No Kontak Pendamping', 'Status Sinkronisasi', 'Nilai', 'Created Date', 'Aksi']


Daftar Sekolah Unik:
  1. Batari School
  2. SD IT NURUL ILMI
  3. SD LENTERA NUSA
  4. SD Swasta Ir.Djuanda Tebing Tinggi
  5. SD santa Melania Sarudik
  6. SDS AL IKHLAS KONGGO
  7. SDS SWASTA METHODISTAN
  8. SMA ISLAM ULUNNUHA
  9. SMA ISLAM ULUNNUHA MEDAN
 10. SMA Negeri 1 Kabanjahe
 11. SMA SWASTA SUTOMO 1 MEDAN
 12. SMAS ISLAM ULUN NUHA
 13. SMAS Islam Ulun Nuha Medan
 14. SMAS ULUN NUHA
 15. SMAS WAGE RUDOLF SUPRATMAN 1 MEDAN
 16. SMAS Wage Rudolf Supratman 2 Medan
 17. SMK TELKOM 1 MEDAN
 18. SMP ISLAM ULUNNUHA
 19. SMP NEGERI 4 KABANJAHE
 20. SMP Swasta Sutomo 1 Medan
 21. SMP swasta Methodist pancur batu
 22. SMPS WAGE RUDOLF SUPRATMA

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


--- Proses Selesai ---
Folder PDF telah diunduh sebagai file zip.
